In [2]:
from typing import List, Dict, Tuple, Union, Any

class IntervalTree:
    def __init__(self, left, right, mid, intervals):
        self.left = left
        self.right = right
        self.mid = mid
        self.intervals = intervals
        
        self.intervals_leftsorted = sorted(intervals, key=lambda x: x[0])
        self.intervals_rightsorted = sorted(intervals, key=lambda x: x[1])
    
    def height(self):
        def height_rec(t):
            if not t:
                return 0
            else:
                return 1 + max(height_rec(t.left), height_rec(t.right))
        return height_rec(self)
    
    def print_tree(self, width=64):
        height = self.height()
        nodes  = [(self, 0)]
        prev_level = 0
        repr_str = ''
        while nodes:
            n,level = nodes.pop(0)
            if prev_level != level:
                prev_level = level
                repr_str += '\n'
            if not n:
                if level < height-1:
                    nodes.extend([(None, level+1), (None, level+1)])
                repr_str += '{val:^{width}}'.format(val='-', width=width//2**level)
            elif n:
                if n.left or level < height-1:
                    nodes.append((n.left, level+1))
                if n.right or level < height-1:
                    nodes.append((n.right, level+1))
                repr_str += '{val:^{width}}'.format(val=str(n.intervals) + ", " + str(n.mid), width=width//2**level)
        print(repr_str)
        
    def search(self, x):
        if x == self.mid:
            return self.intervals
        if x < self.mid:
            # search in left subtree
            intervals = []
            if self.left:
                intervals += self.left.search(x)
            for i in self.intervals_leftsorted:
                if i[0] <= x:
                    intervals.append(i)
                else:
                    break
            return intervals
        if x > self.mid:
            intervals = []
            if self.right:
                intervals += self.right.search(x)
            for i in self.intervals_rightsorted[::-1]:
                if i[1] >= x:
                    intervals.append(i)
                else:
                    break
            return intervals

        
def construct_interval_tree(intervals: List[Tuple]):
    left, right = min([x[0] for x in intervals]), max([x[1] for x in intervals])
    mid = (left + right) // 2
    overlapping_intervals = [x for x in intervals if (x[0] <= mid and x[1] >= mid)]
    left_intervals = [x for x in intervals if x[1] < mid]
    right_intervals = [x for x in intervals if x[0] > mid]
    left_subtree = None
    right_subtree = None
    if left_intervals:
        left_subtree = construct_interval_tree(left_intervals)
    if right_intervals:
        right_subtree = construct_interval_tree(right_intervals)
    return IntervalTree(left_subtree, right_subtree, mid, overlapping_intervals)

In [88]:
test_tree = construct_interval_tree([(0, 3), (4, 9), (0, 1), (2, 3), (4, 7), (8, 9), (4, 6), (4, 5), (0, 9)])
test_tree.print_tree()

          [(4, 9), (4, 7), (4, 6), (4, 5), (0, 9)], 4           
      [(0, 3), (0, 1)], 1                 [(8, 9)], 8           
       -          [(2, 3)], 2          -               -        


In [89]:
test_tree.search(4)

[(4, 9), (4, 7), (4, 6), (4, 5), (0, 9)]

In [18]:
import bisect

def exclude_intervals(interval, excluded: List[Tuple]):
    l = interval[0]
    r = interval[1]
    excluded = sorted(excluded)
    new_intervals = []
    start = bisect.bisect_left([x[0] for x in excluded], l)
    end = bisect.bisect_left([x[0] for x in excluded], r)
    # discard intervals that do not overlap with the input interval
    excluded = excluded[start:end]
    # trim the starting and ending points of the intervals
    if excluded[0][0] < l:
        excluded[0][0] = l 
    if excluded[-1][1] > r:
        excluded[-1][1] = r
    # include the first gap, if any
    if l < excluded[0][0]:
        new_intervals.append((l, excluded[0][0]))
    curr = excluded[0][1]
    for x in excluded[1:]:
        # there is a gap between the current position and the lhs
        # of the next interval
        if curr < x[0]:
            new_intervals.append((curr, x[0]))
            curr = x[1]
        elif curr < x[1]:
            curr = x[1]
    # include the last gap, if any
    if r > excluded[-1][1]:
        new_intervals.append((excluded[-1][1], curr))
    return new_intervals

In [19]:
exclude_intervals((0, 9), [(2, 3), (0, 2), (3, 3)])

[]

In [ ]:
def delete_blocks_from_interval(range_start, range_end, blocks):
    blocks = sorted(blocks)

    # check if the interval overlaps with blocks,
    # if so, truncate the block lists, reset end points if required
    start_idx = bisect.bisect_left([b[0] for b in blocks],range_start)
    end_idx = bisect.bisect_left([b[0] for b in blocks],range_end)
    blocks = blocks[start_idx:end_idx]
    if blocks[0][0] < range_start:
        blocks[0][0] = range_start 
    if blocks[-1][1] > range_end:
        blocks[-1][1] = range_end

    # emit the first gap, if any
    if range_start < blocks[0][0]:
        yield (range_start, blocks[0][0])

    # loop through till the end of the blocks
    end = blocks[0][1]
    for block in blocks[1:]:
        if end < block[0]:
            yield (end, block[0])
            end = block[1]
        elif end < block[1]:
            end = block[1]

    # emit the last gap, if any
    if range_end > blocks[-1][1]:
        yield (blocks[-1][1], range_end)

blocks = sorted([(9, 11), (9, 12), (9, 13), (7, 13), (7, 21), (2, 21), (0, 21)])
list(delete_blocks_from_interval(0, 25, blocks))

In [50]:
def delete_blocks_from_interval(a, b, blocks):
    sorted_blocks = sorted(blocks)
    for i, (c, d) in enumerate(sorted_blocks):
        if a <= c <= b:
            yield (a, c)
            if d > a:
                a = d
            for (e, f) in sorted_blocks[i + 1:]:
                if e <= a <= f:
                    a = f
                elif e > a:
                    break
    if a <= d <= b:
        yield (d, b)
blocks = sorted([(9, 11), (9, 12), (9, 13), (7, 13), (7, 21), (2, 21), (0, 21)])
list(delete_blocks_from_interval(0, 25, blocks))

[(0, 0)]

In [1]:
%pip install mysql.connector

  Using cached mysql-connector-2.2.9.tar.gz (11.9 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp39-cp39-macosx_12_0_arm64.whl size=247970 sha256=215359a99a6a8b1d6680427dd07d294077f1b1d1dbaa260da166c72238f9a172
  Stored in directory: /Users/kevingaojx/Library/Caches/pip/wheels/7b/14/39/5aad423666e827dfe9a1fbcd111ac17171e7c9865d570780ce
Successfully built mysql.connector
You should consider upgrading via the '/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [88]:
import mysql.connector

db = mysql.connector.connect(
    host="useastdb.ensembl.org",
    user="anonymous",
    database="ensembl_compara_107",
    connect_timeout=28800
)

cursor = db.cursor()

In [89]:
def build_select_query(conditions, columns, table):
    scaffold = "SELECT {} FROM {} WHERE ({});"
    conditions_str = " AND ".join(conditions)
    columns_str = ", ".join(columns)
    return scaffold.format(columns_str, table, conditions_str)

In [93]:
cursor.execute("SET @seqid = (SELECT canonical_member_id FROM gene_member WHERE stable_id = 'ENSG00000182463')");
cursor.execute("SELECT * FROM homology_member WHERE homology_id IN (SELECT hm.homology_id FROM homology_member hm LEFT JOIN homology h ON h.homology_id = hm.homology_id WHERE hm.seq_member_id = @seqid AND description LIKE 'ortholog_one2one') AND seq_member_id <> @seqid;")
one_to_one = cursor.fetchall()
cursor.execute("SELECT * FROM homology_member WHERE homology_id IN (SELECT hm.homology_id FROM homology_member hm LEFT JOIN homology h ON h.homology_id = hm.homology_id WHERE hm.seq_member_id = @seqid AND description LIKE 'ortholog_many2many') AND seq_member_id <> @seqid;")
one_to_many = cursor.fetchall()

In [94]:
print(len(one_to_many))

2


In [47]:
cursor.description

[('node_id', 3, None, None, None, None, 0, 16931),
 ('parent_id', 3, None, None, None, None, 1, 16424),
 ('root_id', 3, None, None, None, None, 1, 16424),
 ('left_index', 3, None, None, None, None, 0, 16417),
 ('right_index', 3, None, None, None, None, 0, 33),
 ('distance_to_parent', 5, None, None, None, None, 0, 1),
 ('seq_member_id', 3, None, None, None, None, 1, 16424)]